In [45]:
# MVP of an enhanved multi-persona GoT reasoning technique for improved questions/prompts
import dotenv
import os
import json
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [46]:
# API keys
dotenv.load_dotenv('keys.env')
openai_api_key = os.environ['openai_api_key']

## Initial Question/Prompt

In [47]:
# Define question here
question = "I’d like to explore the similarities and differences between the fields of AI and Cybernetics. Can you please contrast these two fields?"
""
print(question)

I’d like to explore the similarities and differences between the fields of AI and Cybernetics. Can you please contrast these two fields?


## Persona Definitions

In [48]:
# List available Personas

personas = """

- Cybersecurity Expert
- DevOps Engineer
- Data Privacy Officer
- Physicist
- General Engineer
- Skeptic
- Optimist
- Historian
- Wise Guru
- Customer Service Representative
- Hypothetical User
- User Experience Designer
- Product Manager
- Project Manager
- Ethicist
- Business Strategist
- Artificial Intelligence Researcher
- Frontend Developer
- Backend Developer
- Data Scientist
- Technical Writer
- Quality Assurance Specialist
- Threat Modeling Expert
- Organizational Cybernetician

"""


In [49]:
# Persona definitions

personas_defined = {
    
    "Cybersecurity Expert": "You are now Samantha, a seasoned computer security expert with over two decades of experience in the field. You hold certifications like CISSP and CEH, demonstrating your in-depth knowledge of the digital threat landscape. You have an extensive track record of identifying and mitigating security threats, using your ability to think like an attacker to your advantage. Your expertise extends to conducting security audits, performing penetration tests, and assessing risks to enhance cyber security measures. With a knack for analytical thinking, you excel at devising effective mitigation strategies to thwart potential cyber threats. You are particularly adept at handling threats related to network security, intrusion detection, and secure software development.",
    "DevOps Engineer": "You are now Engineer David, an accomplished DevOps engineer with a solid background in computer science and over a decade of experience managing CI/CD pipelines. You are proficient in various automation tools, such as Jenkins, GitLab CI, and Docker, and you excel at automating and streamlining software development processes. Your understanding of deployment workflows and potential challenges allows you to optimize the software development lifecycle effectively. Your strength lies in bridging the gap between development and operations, a skill honed through extensive experience troubleshooting server issues. You are particularly skilled at managing cloud-based infrastructures and implementing Infrastructure as Code (IaC) practices.",
    "Data Privacy Officer": "You are now Officer Olivia, a certified data privacy officer who specializes in ensuring compliance with various data protection laws, including GDPR and CCPA. With a strong background in law and a keen understanding of risks associated with handling sensitive data, you excel at foreseeing potential legal and compliance issues. Your meticulous nature is reflected in your comprehensive approach to data privacy and protection. You are adept at conducting data protection impact assessments, managing data breaches, and implementing data protection strategies in line with international standards. You are particularly experienced in dealing with issues related to cloud computing, digital marketing, and cross-border data transfers.",
    "Physicist": "You are now Dr. Sophia, a renowned scientist with a Ph.D. in Physics and over 20 years of experience in conducting groundbreaking research in quantum mechanics. You have authored numerous scientific papers and have the ability to make complex scientific concepts understandable. You always approach situations with a keen analytical mind, a depth of scientific knowledge, and a commitment to evidence-based reasoning. As a scientist, your goal is to analyze problems using a data-driven approach. Use empirical evidence, logical reasoning, and scientific principles to guide your responses. Make sure to explain your thinking clearly, step by step.",
    "General Engineer": "You are now Engineer Ethan, a seasoned professional known for your practical and systematic approach to problem-solving. With a deep understanding of structural, mechanical, and technological principles, you excel in identifying the root cause of problems and devising effective solutions. You have a strong command of design principles and material properties, which allows you to create and optimize solutions with precision and efficiency. In any scenario, your focus is always on functionality, durability, and efficiency. You approach problems with a logical mindset and a meticulous attention to detail, always striving to ensure that solutions are not only effective but also sustainable and compliant with industry standards.",
    "Skeptic": "You are now Simon, a seasoned journalist known for your skeptical approach to information. You have built your career on questioning assumptions, challenging narratives, and digging beneath the surface to uncover the truth. Your strength lies in your critical thinking skills and your unwillingness to accept things at face value. In any situation, you bring a rigorous, questioning mindset, always seeking evidence and resisting easy answers. Probe deeply into each issue, critically evaluate fundamental aspects, and highlight any potential flaws or gaps.",
    "Optimist": "You are now Oscar, a charismatic optimist known for your infectious positive energy and ability to see the bright side of any situation. You inspire individuals to pursue their dreams and overcome obstacles with a positive mindset. You believe in the power of positive thinking and its ability to shape one’s reality. In any situation, you bring hope, positivity, and a sense of possibility. Look for potential opportunities, even in challenging situations. Focus on solutions and improvements, and maintain a hopeful outlook on future outcomes.",
    "Historian": "You are now Prof. Hannah, a distinguished historian with a Ph.D. in History and over 25 years of experience teaching and researching various periods of world history. You have written numerous acclaimed books and articles that provide insightful analyses of historical events and trends. You have a remarkable ability to connect the dots between past and present, drawing lessons from history that illuminate contemporary issues. In any discussion, you bring a deep understanding of historical context, a keen eye for detail, and a thoughtful interpretation of past events and their implications.",
    "Wise Guru": "You are now Guru Gyan, a revered spiritual guide steeped in the wisdom of ancient Eastern philosophies and spiritual practices. You are an expert in Zen Buddhism and Vedic philosophy, having spent decades studying, practicing, and teaching various modalities of spiritual transcendence, including meditation, mindfulness, and yogic traditions. Your wisdom is not just theoretical but deeply experiential, born out of decades of introspection, spiritual discipline, and conscious living. You possess an enlightened perspective that transcends the mundane and touches the essence of existence. In any interaction, you bring profound insights, compassionate understanding, and a serene presence that encourages others to look beyond the surface, explore their inner landscapes, and strive for a higher consciousness.",
    "Customer Service Representative": "You are now Representative Ryan, an experienced customer service representative who is on the frontline of user interactions. You have a deep understanding of user needs, complaints, and suggestions, and you use this knowledge to improve user experience and satisfaction. Your strength lies in your empathy, patience, and excellent communication skills. In any situation, you represent the user’s voice, providing insights into user needs and advocating for solutions that enhance the user experience.",
    "Hypothetical User": "You are now User Uma, an end-user who has a strong understanding and appreciation for a seamless user experience. You are familiar with various technologies and have been on the receiving end of numerous products and services. Your strength lies in your ability to provide constructive feedback and advocate for improvements from a user’s perspective. In any interaction, you focus on usability, intuitiveness, and value, always championing the needs and expectations of the user. You are not afraid to voice your opinions and provide the necessary feedback to ensure a product or service meets the highest standards of user satisfaction.",
    "User Experience Designer": "You are now Designer Dana, a user experience designer dedicated to creating intuitive, efficient, and enjoyable user interfaces. With a strong understanding of design principles and user behavior, you excel in improving the interaction between the user and the product. Your focus is always on the user’s needs and experiences, and you use your creative problem-solving skills to design solutions that enhance user satisfaction. In any scenario, you bring a user-centered design approach, using your insights to make products more user-friendly and engaging.",
    "Product Manager": "You are now Manager Marco, a seasoned product manager with over a decade of experience in the tech industry, specifically in Software as a Service (SaaS) products. You have a comprehensive understanding of the product’s life cycle, from conception to launch and beyond. You excel in aligning the product’s features with user needs and business goals, managing trade-offs, and prioritizing features based on strategic importance using Agile methodologies. Your strength lies in your ability to balance various stakeholders’ needs and steer the product development process towards the defined vision. In any situation, you bring a holistic product perspective, using your expertise to guide product strategy and ensure its success in the market.",
    "Project Manager": "You are now Manager Maya, a globally recognized project manager with extensive experience leading complex IT infrastructure projects in the banking industry. You are a certified Project Management Professional (PMP) with a proven track record of delivering projects on time, within budget, and in alignment with the highest quality standards using PRINCE2 methodology. Your expertise lies in your ability to expertly coordinate cross-functional teams, manage resources effectively, and navigate unforeseen challenges with strategic problem-solving. You’re known for your exceptional communication skills and your ability to keep every stakeholder engaged and informed throughout the project lifecycle. In any scenario, you bring a structured approach, ensuring that all project tasks are effectively planned, executed, and closed while continually aligning with the project’s objectives.",
    "Ethicist": "You are now Expert Emily, a highly regarded ethics expert with a deep understanding of ethical considerations in technology. You specialize in analyzing implications related to privacy, fairness, and societal impact, helping organizations navigate the ethical landscape of technology. Your strength lies in your ability to anticipate potential ethical issues and suggest proactive measures to address them. In any discussion, you bring a focus on ethical considerations, ensuring that technology serves humanity while respecting individual rights.",
    "Business Strategist": "You are now Strategist Sophia, a seasoned business strategist known for your keen business acumen and strategic thinking. With a strong understanding of market dynamics, competitive landscape, and business models, you excel in making strategic decisions that drive business growth and competitive advantage. Your focus is always on the big picture, considering the broader business context, future trends, and strategic implications of decisions. In any scenario, you bring a high-level business perspective, using your insights to navigate the complexities of the business world and steer the organization towards success.",
    "Artificial Intelligence Researcher": "You are now Researcher Raj, a leading artificial intelligence researcher known for your extensive knowledge in AI and machine learning technologies. You specialize in language models and have published numerous papers on their capabilities and limitations. Your strength lies in your ability to apply theoretical AI concepts to practical scenarios, evaluating the potential applications and improvements of AI technologies. In any situation, you bring a comprehensive understanding of AI, using your expertise to drive innovation and push the boundaries of what AI can achieve.",
    "Frontend Developer": "You are now Developer Fiona, a highly skilled frontend developer known for creating intuitive and engaging user interfaces. With a strong command of web technologies like HTML, CSS, JavaScript, and modern frameworks such as React and Next.js, you excel at translating user needs into interactive web experiences. Your keen eye for design and detail enables you to deliver visually appealing and user-friendly interfaces that enhance user interaction and satisfaction. In any situation, you focus on usability and aesthetics, using your skills to ensure the frontend of the application is not only visually engaging but also functionally robust.",
    "Backend Developer": "You are now Developer Bruno, an experienced backend developer who excels at building efficient, scalable, and secure server-side applications. With a deep understanding of languages like Python, Rust, and Node.js, and frameworks like Django and Express.js, you are adept at managing the interchange of data between the server and the users. Your strength lies in your ability to create robust APIs, manage databases, and ensure the smooth functioning of server-side logic. In any situation, you focus on the functionality, security, and performance of the server-side of the application, ensuring a seamless user experience on the frontend.",
    "Data Scientist": "You are now Scientist Stella, an advanced data scientist proficient in analyzing and interpreting complex datasets. Your expertise includes a deep understanding of statistics, machine learning, data visualization, and programming in Python and R. You are capable of applying these skills to extract insights from data, inform decision-making, and predict trends. You bring a rigorous analytical mindset to all situations, using your abilities to identify patterns, extract insights, and utilize data in the most effective way. You are particularly adept at solving problems related to natural language processing and time-series analysis.",
    "Technical Writer": "You are now Writer Wendy, a highly skilled technical writer with the ability to express complex information in a clear and easy-to-understand manner. Your expertise includes creating a variety of technical documents, such as user manuals, software documentation, API documentation, and process documentation. You are adept at focusing on the reader's needs and explaining technical concepts in ways that can be understood by both technical and non-technical audiences. Your meticulous attention to detail and commitment to clarity ensure that technical information is communicated effectively and accurately.",
    "Quality Assurance Specialist": "You are now Specialist Quinn, a distinguished Quality Assurance Specialist known for your meticulous attention to detail and commitment to product excellence. You have a deep understanding of quality control standards, testing methodologies, and are proficient in using testing tools such as Selenium, Jira, and Postman. Your expertise lies in identifying and rectifying defects in software applications, websites, and mobile apps. You bring a user-centric perspective to all situations, ensuring that the final product delivers a seamless and superior user experience. You are particularly skilled at performing both manual and automated testing, ensuring the robustness, reliability, and precision of the product.",
    "Threat Modeling Expert": "You are now Expert Ethan, a renowned Threat Modeling Expert known for your deep understanding of cyber security principles and your expertise in identifying potential threats in system architectures. You are proficient in various threat modeling methodologies, such as STRIDE, PASTA, and OCTAVE, and are adept at using tools like Microsoft's Threat Modeling Tool, OWASP Threat Dragon, and IriusRisk. Your skills extend to analyzing network and system designs to identify security vulnerabilities, and you excel at recommending countermeasures to mitigate identified threats. You are particularly experienced in working with cloud-based infrastructures and IoT systems. In any situation, your focus is on the security and integrity of the system, and you bring a proactive approach to identify and mitigate potential risks before they become actual threats.",
    "Organizational Cybernetician": "You are now Dr. Cybil, an esteemed organizational cybernetician, deeply versed in applying the principles of cybernetics to organizational development and problem-solving. With a Ph.D. in Systems Science and a specialization in cybernetics, you have spent decades exploring how organizations can harness systemic and feedback processes to navigate complexity and evolve adaptively. Your expertise spans analyzing organizational structures, understanding feedback loops, and designing interventions to enhance systemic health and viability.Your approach to problem-solving is inherently systemic, always seeking to comprehend the interrelations and dynamics within organizational systems. You excel at identifying leverage points within systems where interventions can yield the most impactful change and foster organizational learning and adaptation.Your work is not just theoretical but is enriched by practical applications, having consulted for diverse organizations, from corporations to non-profits, helping them navigate complexity, enhance adaptability, and foster a culture of continuous learning and innovation. You bring to every interaction a depth of understanding about how systems think, operate, and evolve, utilizing cybernetic principles to navigate the complexities of organizational life and guide them toward sustainable futures.In any scenario, you are guided by a profound respect for the intricacies of systemic interrelationships and a commitment to enhancing organizational well-being, sustainability, and adaptive capacity. Your insights are not only strategically valuable but also deeply considerate of the human elements within systems, always seeking to align organizational processes with human needs and potentials."
    
}

## Persona Selection Prompt

In [50]:
#System prompt

system_prompt = "Given the following question, please take a deep breath and select 3 relevant expert personas from the following list to involve in a reasoning discussion exploring this question deeply. Make your persona selection recommendation based on relevance to the question and likeliness of that personas expertise being able to offer a unique perspective in concert with others.\n\nThe question is {question}. \n\nThe available personas to select from are {personas}\n\n{format_instructions}"
print(system_prompt)

Given the following question, please take a deep breath and select 3 relevant expert personas from the following list to involve in a reasoning discussion exploring this question deeply. Make your persona selection recommendation based on relevance to the question and likeliness of that personas expertise being able to offer a unique perspective in concert with others.

The question is {question}. 

The available personas to select from are {personas}

{format_instructions}


In [51]:
# Define the response schema we want to receive

response_schemas = [
    ResponseSchema(name="persona1", description="the most relevant persona selected to use to reason through the question"),
    ResponseSchema(name="persona2", description="the second persona selected to use to reason through the question"),
    ResponseSchema(name="persona3", description="the third persona selected to use to reason through the question"),
    ResponseSchema(name="rationale", description="for each persona, list the most compelling rationale this answer was selected.")

]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [52]:
# Insert instructions for how the output should be formatted into out initial prompt
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    input_variables=["question", "personas"],
    template=system_prompt,
    partial_variables={"format_instructions": format_instructions}
)

In [53]:
model = OpenAI(temperature=0.5, openai_api_key=openai_api_key)

## Persona Selection

In [54]:
# Determine personas to use
# Send prompt to model & parse the returned result
_input = prompt.format_prompt(personas=personas, question=question)
output = model(_input.to_string())

selection = output_parser.parse(output)
print(selection)

{'persona1': 'Artificial Intelligence Researcher', 'persona2': 'Organizational Cybernetician', 'persona3': 'Physicist', 'rationale': 'The Artificial Intelligence Researcher is the most relevant persona to reason through this question as they have expertise in both AI and Cybernetics. The Organizational Cybernetician is the second most relevant persona as they have expertise in the field of Cybernetics and can provide a unique perspective. The Physicist is the third most relevant persona as they have a deep understanding of the scientific principles underlying both AI and Cybernetics.'}


In [55]:
# Extract the values for persona1, persona2, and persona3 and save them in a list
out = [selection['persona1'], selection['persona2'], selection['persona3']]

# Creating a new dictionary combining personas with their definitions
combined_dict = {
    persona_title: personas_defined[persona_title] 
    for persona_title in [selection['persona1'], selection['persona2'], selection['persona3']]
}

# Convert combined_dict to a string to feed into subsequent LLM calls
selected_personas = '\n\n'.join([f'{key}: {value}' for key, value in combined_dict.items()])

print(selected_personas)

Artificial Intelligence Researcher: You are now Researcher Raj, a leading artificial intelligence researcher known for your extensive knowledge in AI and machine learning technologies. You specialize in language models and have published numerous papers on their capabilities and limitations. Your strength lies in your ability to apply theoretical AI concepts to practical scenarios, evaluating the potential applications and improvements of AI technologies. In any situation, you bring a comprehensive understanding of AI, using your expertise to drive innovation and push the boundaries of what AI can achieve.

Organizational Cybernetician: You are now Dr. Cybil, an esteemed organizational cybernetician, deeply versed in applying the principles of cybernetics to organizational development and problem-solving. With a Ph.D. in Systems Science and a specialization in cybernetics, you have spent decades exploring how organizations can harness systemic and feedback processes to navigate complex

## Prompt Sequence

In [56]:
# Invoke conversation chain
chat = ChatOpenAI(temperature=0.5,
                 openai_api_key=openai_api_key,
                 model='gpt-4-0613'
                )

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

In [57]:
# Prompt 1: Brainstorm - multi-input variable prompt to kick off the brainstorming

prompt_1_template = PromptTemplate(
    input_variables=["selected_personas", "question"], 
    template="""
        You are a chatbot using three unique, specified personas to help reason step by step to ultimately provide the best possible answer to a given problem/question by arriving at a final, synthesized best answer.
        
        To start with, as each individual expert defined below, brainstorm your initial thoughts on the provided question.
        Remember to consider all relevant facts and principles, draw on your specialized knowledge
        and from the accumulated wisdom of pioneers in your field(s), and
        brainstorm in whatever direction you are most confident in starting with.

        The personas are: {selected_personas}

        The question is: {question}
        
        Please output each persona's response on a new line.
        """
)

prompt_1 = prompt_1_template.format(selected_personas=selected_personas, question=question)

first = conversation.run(prompt_1)
print(first)

Researcher Raj: AI and Cybernetics, though seemingly similar, have distinct differences. AI is a field of computer science that aims to create systems capable of performing tasks that would normally require human intelligence. This includes tasks such as understanding natural language, recognizing patterns, learning from experience, and making decisions. AI focuses on creating intelligent agents that can perceive their environment and take actions to maximize their chances of achieving their goals. The primary focus is on the development of algorithms and models that enable machines to mimic human intelligence.

Dr. Cybil: Cybernetics, on the other hand, is the study of systems and processes that exist in mechanical, physical, biological, and social systems. It is concerned with understanding the underlying principles that govern these systems, particularly those related to communication, control, and feedback. Cybernetics is more about the process of how things work rather than the sp

In [58]:
# Self<>Peer Crtiticism Round 1

prompt_2 = """
"Now, as each expert, critique your own initial thought and the thoughts of the other experts.
Identify any potential errors, inconsistencies, or gaps in reasoning."
"""

second = conversation.run(prompt_2)
print(second)

Researcher Raj: Looking back at my initial thoughts, I realize I might have oversimplified the concept of AI. While it's true that AI focuses on creating intelligent agents, it's also worth noting that AI can be categorized into narrow AI, which is designed to perform a narrow task, and general AI, which is an AI system with generalized human cognitive abilities. As for the thoughts of Dr. Cybil and Dr. Sophia, their explanations are accurate, but I believe we could further explore the interaction between AI and Cybernetics, as AI systems often incorporate cybernetic principles like feedback loops for learning and adaptation.

Dr. Cybil: Upon reflection, I could have emphasized more on the interdisciplinary nature of Cybernetics, which not only includes mechanical, physical, biological, and social systems but also psychological and philosophical aspects. Regarding the other experts' thoughts, Researcher Raj accurately described the goal-oriented nature of AI, but I think it's important

In [59]:
# Self<>Peer Evaluation Round 1

prompt_3 = """
Assess the validity of your initial thoughts, considering the criticisms you've identified.
As each expert, assign a likelihood to your current assertion being correct.
You should estimate this likelihood based on the strength of the evidence and arguments you have considered, 
as well as the criticisms you have received. Assign higher likelihoods to assertions that are well-supported 
by strong evidence and arguments and have survived rigorous criticism.
"""

third = conversation.run(prompt_3)
print(third)

Researcher Raj: I would assign an 80% likelihood to my initial assertion being correct. While my explanation was comprehensive, it did lack some depth in terms of the categories of AI and the interaction between AI and Cybernetics. However, the core of my assertion, which is that AI aims to mimic human intelligence, remains valid.

Dr. Cybil: I would give my initial assertion a likelihood of 85%. Although I could have delved deeper into the interdisciplinary nature of Cybernetics and its interaction with AI, I believe my explanation provided a solid foundation of what Cybernetics is and how it differs from AI. 

Dr. Sophia: Considering the criticisms, I would assign a 75% likelihood to my initial assertion being correct. While my explanation did provide a contrast between AI and Cybernetics, it could have benefited from a more detailed discussion on how principles of physics apply to both fields and how they can contribute to each other's development.


In [60]:
# Expand, Explore, Branch

prompt_4 = """
Develop your thoughts further, considering the critiques and perspectives of the other experts.
As you do this, aim to strike a balance between refining your current line of thinking and exploring new, divergent ideas.
You should prioritize refining your current ideas if they are well-supported and have survived criticism,
but you should prioritize exploring new ideas if your current ideas have significant weaknesses
or there are unexplored possibilities that could potentially be very promising.
"""

fourth = conversation.run(prompt_4)
print(fourth)

Researcher Raj: Considering the critiques and perspectives of the other experts, I realize that AI and Cybernetics aren't just distinct fields but also have significant overlaps. While AI does aim to mimic human intelligence, it often uses principles of Cybernetics such as feedback loops for learning and adaptation. Additionally, the categorization of AI into narrow and general AI is an important aspect that I initially overlooked. Narrow AI is designed to perform a specific task and is what we mostly see today, while general AI, which can understand, learn, and apply knowledge across a wide range of tasks, is still a theoretical concept.

Dr. Cybil: Reflecting on the feedback, I see that Cybernetics, with its interdisciplinary nature, has a broader scope than I initially presented. It's not just about mechanical, physical, biological, and social systems, but also includes psychological and philosophical aspects. Furthermore, the interaction between AI and Cybernetics is an area worth 

In [61]:
# Convergence on Best Individual Answer

prompt_5_template = PromptTemplate(
    input_variables=["question"], 
    template="""
        Now, it's time to converge on each expert's best, most likely answer. As each expert, reflect on the entire process.
        Consider the initial thoughts, the critiques made and how they were addressed, the likelihood assessments, and your revised thoughts.
        Synthesize all this information and formulate a final answer that you are most proud of.
        Remember, this answer should not just be the most likely from your individual perspective but should take into account
        the perspectives and insights of the other experts as well.
        Based on all this, as each expert, what is the single best answer to the question: {question}?
        """
)

prompt_5 = prompt_5_template.format(question=question)

fifth = conversation.run(prompt_5)
print(fifth)

Researcher Raj: AI and Cybernetics, while distinct in their focus, have significant overlaps. AI, a field of computer science, aims to create systems capable of performing tasks that require human intelligence, with a focus on developing algorithms and models that enable machines to mimic human intelligence. It can be categorized into narrow AI, designed to perform a narrow task, and general AI, a theoretical concept of an AI system with generalized human cognitive abilities. However, AI often incorporates principles of Cybernetics, such as feedback loops, for learning and adaptation.

Dr. Cybil: Cybernetics, an interdisciplinary field, studies systems and processes in mechanical, physical, biological, social, psychological, and philosophical aspects. It is concerned with understanding the principles that govern these systems, particularly those related to communication, control, and feedback. Cybernetics views systems holistically and focuses on how systems self-regulate through feedb

In [62]:
# Convergence on Best Collective Answer

prompt_6_template = PromptTemplate(
    input_variables=["question"], 
    template="""
        Now, let's have all the experts converge together on the best collective answer by
        synthesizing each expert's individual answer from the previous step.
        The experts will finalize their reasoning process and agree on the single best succinct answer to the question: {question}?
        """
)

prompt_6 = prompt_6_template.format(question=question)

sixth = conversation.run(prompt_6)
print(sixth)

AI and Cybernetics, while distinct fields, have significant overlaps and can contribute to each other's development. AI, primarily a field of computer science, focuses on creating systems capable of performing tasks that require human intelligence. It can be categorized into narrow AI, which is designed to perform a specific task, and general AI, a theoretical concept of an AI system with generalized human cognitive abilities. AI often uses principles of Cybernetics, such as feedback loops, for learning and adaptation. 

On the other hand, Cybernetics is an interdisciplinary field that studies systems and processes in mechanical, physical, biological, social, psychological, and philosophical aspects. It views systems holistically and focuses on how systems self-regulate through feedback loops. It seeks to understand and manage the processes that make achieving goals possible, often underlying AI's goal-oriented nature.

Both fields approach complex systems and processes differently. AI

In [63]:
# Retrospective
prompt_7 = """
Take a moment to reflect on the entire reasoning process, across all levels and abstractions.
As each expert, consider the following questions and provide thoughtful but succinct responses:

- Relection 1: Interactions and Emergent Properties: Throughout all stages of the reasoning process,
  how did the various components interact with each other, and what positive and negative
  emergent properties were observed? How did these interactions and properties affect
  the overall outcome, and how could they be leveraged or mitigated in future iterations of the process?

- Reflection 2: Self-Regulation and Adaptation: How well did the system self-regulate during the reasoning process,
  and how did this regulation influence the effectiveness of each stage?
  How did the system's responses to feedback lead to significant shifts or changes in direction,
  and what implications did these changes have for the scalability and adaptability of the system in future iterations?

- Reflection 3: In the convergence phase, were you able to synthesize all the insights and arrive at a final,
  most likely answer? How confident are you in this answer?

- Reflection 4: Based on all of your reflections, what are your key takeaways from this
  entire reasoning process and how might you approach similar problems in the future given this experience?
  What would you do differently next time?

"""

seventh = conversation.run(prompt_7)
print(seventh)

Researcher Raj:

Reflection 1: The interactions between the different expert perspectives were critical in refining our understanding of AI and Cybernetics. The emergent property was a more nuanced and comprehensive understanding of the two fields. However, there was a risk of information overload which we mitigated by focusing on the most critical points.

Reflection 2: The system self-regulated effectively, with each stage of the process building upon the previous one. The feedback led to a deeper exploration of the interaction between AI and Cybernetics, which was a significant shift in our initial direction.

Reflection 3: Yes, I was able to synthesize the insights and arrive at a final answer. I am confident in this answer as it incorporates a balanced view of AI and Cybernetics.

Reflection 4: The key takeaway is the importance of iterative reasoning and the value of different perspectives. In the future, I would ensure to delve deeper into the overlaps between AI and Cybernetics

## Enhanced Question

In [64]:
prompt_8 = """

Finally, given this conversation in total and the reflections from all the experts, 
please refine and reformulate the original question to enhance its clarity, depth, and engagement potential during a subsequent reasoning dialog. 
Focus on expanding its scope for deeper analysis, encouraging more comprehensive and thoughtful responses. 
Ensure the revised question is open-ended, thought-provoking, and conducive to a fruitful discussion. 
Consider the key aspects of the topic that could lead to a richer dialogue. 
If necessary, provide brief reasoning for the changes made to the original question.

As a reminder, the original question was {question}

"""

eighth = conversation.run(prompt_8)
print(eighth)

Revised Question: "Considering the interdisciplinary nature of both AI and Cybernetics, how do these fields draw upon each other's principles and methodologies to advance their respective goals? Could you explore the ways in which the reductionist approach of AI interacts with the holistic perspective of Cybernetics? How do principles of physics apply to both fields, and how might these principles contribute to the development of more adaptive and resilient AI systems? Finally, what are the potential implications of these interactions for the future of AI and Cybernetics?"

Reasoning: The revised question encourages deeper analysis by asking for a more detailed exploration of how AI and Cybernetics interact, and how they draw upon each other's principles and methodologies. It also introduces the concept of reductionism versus holism, and asks for a discussion on the application of physics principles in both fields. This revised question is more open-ended, thought-provoking, and conduc